In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [55]:
R = tf.Variable(0.5, trainable = False)
center = tf.Variable([[0.,0.,0.]], trainable = False)

x = tf.ones((10,1)) * -5
y = tf.random.uniform((10,1), -5, 5)
z = tf.zeros((10,1))
psi = tf.ones((10,1)) * np.pi/2
theta = tf.random.uniform((10,1), -np.pi/2, np.pi/2)

init_pos = tf.concat((x,y,z,psi,theta), axis=1)

def spherical_to_vec(inputs):
    x, y, z, psi, theta = tf.split(inputs, 5, axis=1)
    r_x = np.sin(psi) * np.cos(theta)
    r_y = np.sin(psi) * np.sin(theta)
    r_z = tf.cos(theta)
    return tf.concat((x, y, z, r_x, r_y, r_z), axis=1)

input_data = spherical_to_vec(init_pos)

def PINN():
        model = tf.keras.Sequential([
            tf.keras.Input(shape=(6,)),
            tf.keras.layers.Dense(10, activation='relu', kernel_initializer='glorot_normal', name='hidden1'),
            tf.keras.layers.Dense(10, activation='relu', kernel_initializer='glorot_normal', name='hidden2'),
            tf.keras.layers.Dense(1, name='output')
        ])
        return model
    
PINN_list = [PINN() for _ in range(10)]

def prop_(inputs):
    x, y, z, r_x, r_y, r_z, t = tf.split(inputs, 7, axis=1)
    x_prop = x + r_x * t
    y_prop = y + r_y * t
    z_prop = z + r_z * t
    return tf.concat((x_prop,y_prop,z_prop), axis=1)

def intersection(inputs):
    inter_ = tf.square(tf.reduce_sum(tf.square(inputs- center), keepdims=True) - tf.square(R)) 
    return inter_

def get_loss(PINN_list, input_data):
    
    pred_t = [PINN_list[i](input_unit) for i, input_unit in enumerate(tf.split(input_data,10))]
    
    total_inp = [tf.concat([tf.reshape(input_unit,(1,-1)), t], axis=1) for input_unit, t in zip(input_data, pred_t)]
    
    prop = [prop_(inp) for inp in total_inp]
    
    inter_loss = [intersection(prop_unit) for prop_unit in prop]
    
    prop_x_loss = [tf.math.maximum(-prop_unit[:,0:1], 0) for prop_unit in prop]
    
    total_loss = [inter + prop_x for inter, prop_x in zip(inter_loss, prop_x_loss)]
    
    return total_loss

lr = 1e-2
optim = tf.keras.optimizers.Adam(learning_rate=lr)

num_epochs = 500
for epoch in range(num_epochs):
    with tf.GradientTape(persistent=True) as tape:
        trainable = [PINN_list[i].trainable_weights for i in range(10)]

        loss = get_loss(PINN_list, input_data)
    g = tape.gradient(loss, trainable)
    optim.apply_gradients(zip(g, trainable))

AttributeError: 'list' object has no attribute '_unique_id'